In [7]:
using SparseArrays
using Zygote
using Flux: Optimise

In [2]:
import Base: broadcast, broadcasted

function broadcast(-, arr_sink::AbstractSparseMatrix{Tv,Ti}, arr_source::AbstractSparseMatrix{Tv,Ti}) where {Tv,Ti}
    for (col_index, (ptr_sink, end_sink)) in enumerate(zip(arr_sink.colptr[1:end-1], arr_sink.colptr[2:end]))
        ptr_source = arr_source.colptr[col_index]
        end_source = arr_source.colptr[col_index + 1]
        while ptr_sink < end_sink && ptr_source < end_source
            #println(ptr_sink, " ", arr_sink.rowval[ptr_sink], " ", ptr_source, " ", arr_source.rowval[ptr_source])
            if arr_sink.rowval[ptr_sink] < arr_source.rowval[ptr_source]
                #println("sink up")
                ptr_sink += 1
                continue
            end
            if arr_sink.rowval[ptr_sink] > arr_source.rowval[ptr_source]
                #println("source up")
                ptr_source += 1
                continue
            end
            row_index = arr_sink.rowval[ptr_sink]
            #println(row_index, " ", col_index, " ", arr_sink.nzval[ptr_sink], " ", arr_source.nzval[ptr_source])
            arr_sink.nzval[ptr_sink] -= arr_source.nzval[ptr_source]
            ptr_sink += 1
            ptr_source += 1
        end
    end
    return arr_sink
end

function broadcasted(-, arr_sink::AbstractSparseMatrix{Tv,Ti}, arr_source::AbstractSparseMatrix{Tv,Ti}) where {Tv,Ti}
    broadcast(-, arr_sink, arr_source)
end

broadcasted (generic function with 94 methods)